In [1]:
!cat /proc/cpuinfo

# cpu가 Intel(R) Xeon(R) CPU @ 2.30GHz이어야함

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [ ]:
!git clone https://github.com/Team-M1/badwords-classifier-train
%cd badwords-classifier-train
!pip install -r requirements.txt

# 모델 불러오기

각자 알아서 불러올 것

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained("대충 모델")
tokenizer = AutoTokenizer.from_pretrained("대충 토크나이저")

In [ ]:
# 테스트 데이터

test_df = pd.read_csv("data/test.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(100)

## 스피드 테스트

In [ ]:
model.to("cpu")
model.eval()
print()

In [ ]:
%%timeit

with torch.no_grad():
    tokens = tokenizer(list(speed_test_data), padding="max_length", truncation=True, return_tensors="pt")
    _ = model(**tokens)

## F1 스코어

In [ ]:
# f1
model.cuda()

with torch.no_grad():
    tokens = tokenizer(list(test_data), padding="max_length", truncation=True, return_tensors="pt")
    tokens = tokens.to("cuda")
    preds = model(**tokens)

In [ ]:
from sklearn.metrics import f1_score


logits = preds.logits.cpu().numpy()
pred = np.argmax(logits, 1)

f1 = f1_score(test_label, pred, average="macro")
print(f1)